# Example for creating a custom process in AeroMAPS

This document aims to show the recommended way to create a custom process (configuration file and custom models) and execute them within AeroMAPS.

## Load and process

First, the user has to load the framework and generate a process.

In [ ]:
%matplotlib widget
from aeromaps import create_process

import gemseo as gm
from aeromaps.utils.functions import custom_logger_config
custom_logger_config(gm.configure_logger())

## Create a custom model

By default, AeroMAPS loads standard models from its internal library. These models are organized into categories (traffic, fleet, operations, energy, etc.) and can be enabled or disabled via the configuration file.

Custom models can be added to extend AeroMAPS functionality. We create a custom model example `MeanDistancePerInhabitantFlyer` that is located [here](./models/mean_distance_per_inhabitant_and_flyer.py), in the file `models/mean_distance_per_inhabitant_and_flyer.py`. 

To integrate it into the AeroMAPS process, we specify it in the configuration file under `models.customs`:

```yaml
models:
  customs:
    MeanDistancePerInhabitantFlyer: "./models/mean_distance_per_inhabitant_and_flyer.py"
```

> **Note:** The path is relative to the configuration file location. The class name can be inferred from the model name, or explicitly specified using the format `"./path/to/file.py::ClassName"`.

## Create the process

Here we show how we can use a configuration file with an example [here](./data/config.json), in the file `data/config.json`.
There you can specify the relative path to the different files.
For example, you can provide a custom input file with the parameters you want to modify with an example [here](./data/inputs.json), in the file `data/inputs.json`.

Be careful, do not forget to set the inputs required by your models, either in the `resources/data/parameters.json` file if you modify the source code, or directly in the `inputs.json` file from your configuration file. You can also change the inputs data directly through the code after the creation of the process (see below).

In [ ]:
process = create_process(configuration_file="data/config.yaml")

In [ ]:
process.parameters.world_inhabitant_number_reference_years = [2020, 2030, 2040, 2050]
process.parameters.world_inhabitant_number_reference_years_values = [
    7.805e9,
    8.512e9,
    9.159e9,
    9.687e9,
]  # Assumption based on the 2022 Revision of World Population Prospects from the United Nations
process.parameters.inhabitant_flyer_share_reference_years = [2020, 2030, 2040, 2050]
process.parameters.inhabitant_flyer_share_reference_years_values = [
    11,
    14,
    17,
    20,
]  # Assumption based on a linear increase from Gossling and Humpe (2020) value

## Compute

Once all the parameters have been set up, the user can compute.

In [ ]:
process.compute()
process.write_json()

## Results

The user can then display the results. The user has access to float outputs but also to annual data outputs, with the possibility of choosing the output.

In [ ]:
process.data["vector_outputs"][["mean_distance_per_inhabitant", "mean_distance_per_flyer"]]

In [ ]:
# Verify the outputs between .outputs.json and data/reference/outputs.json
from aeromaps.utils.functions import compare_json_files

files_are_different = compare_json_files(
    "./data/reference/outputs.json",
    "./data/outputs.json",
    rtol=0.0001,
    atol=0,
)

if files_are_different:
    raise ValueError("The outputs.json files are different.")

In [ ]:
from aeromaps.utils.functions import clean_notebooks_on_tests

clean_notebooks_on_tests(globals(), force_cleanup=False)